<a href="https://colab.research.google.com/github/vdhinaka/PythonDeepDive/blob/master/Sequences/06%20-%20Custom%20Sequences%20-%20Part%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Custom Sequences (Part 1)
An immutable sequence type should support two things:
- returning the length of the sequence (\__len__)
- given an index, returning the element at that index (\__getitem__)  - return an element for a valid index or raise an IndexError exception for an invalid index

We always know the index of the first element of the sequence. <br>
We can choose to support negative indices: 
- i < 0 --> i = length - i

We can choose to support slicing:
- accept slice objects as argument to \__getitem__









We'll focus first on how to create a custom sequence type that supports indexing, slicing (read only) and iteration. We'll look into mutable custom sequences in an upcoming video.

First we should understand how the `__getitem__` method works for iteration and retrieving individual elements from a sequence:

In [ ]:
my_list = [0, 1, 2, 3, 4, 5]

In [ ]:
my_list.__getitem__(0)

0

In [ ]:
my_list.__getitem__(5)

5

But if our index is out of bounds:

In [ ]:
my_list.__getitem__(6)

IndexError: list index out of range

we get an IndexError.

Technically, the `list` object's `__getitem__` method also supports negative indexing and slicing:

In [ ]:
my_list.__getitem__(-1)

5

In [ ]:
my_list.__getitem__(slice(0,6,2))

[0, 2, 4]

In [ ]:
my_list.__getitem__(slice(None, None, -1))

[5, 4, 3, 2, 1, 0]

#### Mimicking Python's `for` loop using the `__getitem__` method

In [ ]:
my_list = [0, 1, 2, 3, 4, 5]

In [ ]:
for item in my_list:
    print(item ** 2)

0
1
4
9
16
25


Now let's do the same thing ourselves without a for loop:

In [ ]:
index = 0
while True:
    try:
        item = my_list.__getitem__(index)
    except IndexError:
        # reached the end of the sequence
        break
    # do something with the item...
    print(item ** 2)
    index += 1

0
1
4
9
16
25


#### Implementing a custom Sequence 

Custom objects can support slicing - we'll see this later in this course, but for now we'll take a quick peek ahead.

To make a custom classes support indexing (and slicing) we only need to implement the `__getitem__` method which receives the index (or slice) we are interested in.

In [ ]:
class MySequence:
    def __getitem__(self, index):
        print(type(index), index)

In [ ]:
my_seq = MySequence()

In [ ]:
my_seq[0]

<class 'int'> 0


In [ ]:
my_seq[100]

<class 'int'> 100


In [ ]:
my_seq[0:2]

<class 'slice'> slice(0, 2, None)


In [ ]:
my_seq[0:10:2]

<class 'slice'> slice(0, 10, 2)


As you can see, the `__getitem__` method receives an index number of type `int` when we use `[n]` and a `slice` object when we use `[i:j]` or `[i:j:k]`.

As we saw in a previous lecture, given the bounds for a slice, and the length of the sequence we are slicing, we can always define a `range` that will generate the desired indices.

We also saw that the `slice` object has a method, `indices`, that precisely tells us the start/stop/step values we would need for an equivalent `range`, given the length of the sequence we are slicing.

Let's recall a simple example first:

In [ ]:
l = 'python'
len(l)

6

In [ ]:
s = slice(0, 6, 2)
l[s]

'pto'

In [ ]:
s.start, s.stop, s.step

(0, 6, 2)

In [ ]:
s.indices(6)

(0, 6, 2)

In [ ]:
list(range(0, 6, 2))

[0, 2, 4]

This matches exactly the indices that were selected from the sequence `'python'`

### Example

So, why am I re-emphasizing this equivalence between the indices in a `slice` and and equivalent `range` object?

Let's say we want to implement our own sequence type and we want to support slicing.

For this example we'll create a custom Fibonacci sequence type.

First recall that the `__getitem__` will receive either an integer (for simple indexing), or a slice object:

In [ ]:
class Fib:
    def __getitem__(self, s):
        print(type(s), s)

In [ ]:
f = Fib()
f[2]
f[2:10:2]

<class 'int'> 2
<class 'slice'> slice(2, 10, 2)


We'll use that to implement both indexing and slicing for our custom Fibonacci sequence type.

We'll make our sequence type bounded (i.e. we'll have to specify the size of the sequence). But we are not going to pre-generate the entire sequence of Fibonacci numbers, we'll only generate the ones that are being requested as needed.

In [ ]:
class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            print(f'requesting [{s}]')
        else:
            # slice being requested
            print(f'requesting [{s.start}:{s.stop}:{s.step}]')

In [ ]:
f = Fib(10)

In [ ]:
f[3]

requesting [3]


In [ ]:
f[:5]

requesting [None:5:None]


Let's now add in what the equivalent range would be:

In [ ]:
class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            print(f'requesting [{s}]')
        else:
            # slice being requested
            print(f'requesting [{s.start}:{s.stop}:{s.step}]')
            idx = s.indices(self._n)
            rng = range(*idx)
            print(f'\trange({idx[0]}, {idx[1]}, {idx[2]}) --> {list(rng)}')

In [ ]:
f = Fib(10)
f[3:5]
f[::-1]

requesting [3:5:None]
	range(3, 5, 1) --> [3, 4]
requesting [None:None:-1]
	range(9, -1, -1) --> [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


Next step is for us to actually calculate the n-th Fibonacci number, we'll use memoization as well (see lecture on decorators and memoization if you need to refresh your memory on that):

In [ ]:
from functools import lru_cache

In [ ]:
@lru_cache(2**10)
def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n-1) + fib(n-2)

In [ ]:
fib(0), fib(1), fib(2), fib(3), fib(4), fib(5), fib(50)

(1, 1, 2, 3, 5, 8, 20365011074)

Now, let's make this function part of our class:

In [ ]:
class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            print(f'requesting [{s}]')
        else:
            # slice being requested
            print(f'requesting [{s.start}:{s.stop}:{s.step}]')
            idx = s.indices(self._n)
            rng = range(idx[0], idx[1], idx[2])
            print(f'\trange({idx[0]}, {idx[1]}, {idx[2]}) --> {list(rng)}')
    
    @staticmethod
    @lru_cache(2**32)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return fib(n-1) + fib(n-2)

The next step is to implement the `__getitem__` method. Let's start by implementing the simple indexing:

In [ ]:
class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            return self._fib(s)
        else:
            # slice being requested
            print(f'requesting [{s.start}:{s.stop}:{s.step}]')
            idx = s.indices(self._n)
            rng = range(idx[0], idx[1], idx[2])
            print(f'\trange({idx[0]}, {idx[1]}, {idx[2]}) --> {list(rng)}')
            
    @staticmethod
    @lru_cache(2**32)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return fib(n-1) + fib(n-2)

Let's test that out:

In [ ]:
f = Fib(100)

In [ ]:
f[0], f[1], f[2], f[3], f[4], f[5], f[50]

(1, 1, 2, 3, 5, 8, 20365011074)

But we still have a few problems.

First we do not handle negative values, and we also will return results for indices that should technically be out of bounds, so we can't really iterate through this sequence yet as we would end up with an infinite iteration!

In [ ]:
f[200], f[-5]

(453973694165307953197296969697410619233826, 1)

So we first need to raise an `IndexError` exception when the index is out of bounds, and we also need to remap negative indices (for example `-1` should correspond to the last element of the sequence, and so on)

In [ ]:
class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            if s < 0:
                s = self._n + s
            if s < 0 or s > self._n - 1:
                raise IndexError
            return self._fib(s)
        else:
            # slice being requested
            print(f'requesting [{s.start}:{s.stop}:{s.step}]')
            idx = s.indices(self._n)
            rng = range(idx[0], idx[1], idx[2])
            print(f'\trange({idx[0]}, {idx[1]}, {idx[2]}) --> {list(rng)}')
            
    @staticmethod
    @lru_cache(2**32)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return fib(n-1) + fib(n-2)

In [ ]:
f = Fib(10)

In [ ]:
f[9], f[-1]

(55, 55)

In [ ]:
f[10]

IndexError: 

In [ ]:
f[-100]

IndexError: 

In [ ]:
for item in f:
    print(item)

1
1
2
3
5
8
13
21
34
55


We still don't support slicing though...

In [ ]:
f[0:2]

requesting [0:2:None]
	range(0, 2, 1) --> [0, 1]


So let's implement slicing as well:

In [ ]:
class Fib:
    def __init__(self, n):
        self._n = n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            if s < 0:
                s = self._n + s
            if s < 0 or s > self._n - 1:
                raise IndexError
            return self._fib(s)
        else:
            # slice being requested
            idx = s.indices(self._n)
            rng = range(idx[0], idx[1], idx[2])
            return [self._fib(n) for n in rng]
            
    @staticmethod
    @lru_cache(2**32)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return fib(n-1) + fib(n-2)

In [ ]:
f = Fib(10)

In [ ]:
f[0:5]

[1, 1, 2, 3, 5]

In [ ]:
f[5::-1]

[8, 5, 3, 2, 1, 1]

In [ ]:
list(f)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [ ]:
f[::-1]

[55, 34, 21, 13, 8, 5, 3, 2, 1, 1]

One other thing, is that the built-in `len` function will not work with our class:

In [ ]:
f = Fib(10)

In [ ]:
len(f)

TypeError: object of type 'Fib' has no len()

That's an easy fix, we just need to implement the `__len__` method:

In [ ]:
class Fib:
    def __init__(self, n):
        self._n = n
    
    def __len__(self):
        return self._n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            # single item requested
            if s < 0:
                s = self._n + s
            if s < 0 or s > self._n - 1:
                raise IndexError
            return self._fib(s)
        else:
            # slice being requested
            idx = s.indices(self._n)
            rng = range(idx[0], idx[1], idx[2])
            return [self._fib(n) for n in rng]
            
    @staticmethod
    @lru_cache(2**32)
    def _fib(n):
        if n < 2:
            return 1
        else:
            return fib(n-1) + fib(n-2)

In [ ]:
f = Fib(10)

In [ ]:
len(f)

10

One thing I want to point out here: we did not need to use inheritance! There was no need to inherit from another sequence type. All we really needed was to implement the `__getitem__` and `__len__` methods.

The other thing I want to mention, is that I would not use recursion for production purposes for a Fibonacci sequence, even with memoization - partly because of the cost of recursion and the limit to the recursion depth that is possible.

Also, when we look at generators, and more particularly generator expressions, we'll see better ways of doing this as well.

I really wanted to show you a simple example of how to create your own sequence types.